<a href="https://bmi.readthedocs.io"><img src="https://raw.githubusercontent.com/csdms/espin/main/media/bmi-logo-header-text.png"></a>

# Run the `Heat` model through its BMI

`Heat` models the diffusion of temperature on a uniform rectangular plate with Dirichlet boundary conditions. This is the canonical example used in the [bmi-example-python](https://github.com/csdms/bmi-example-python) repository. View the source code for the [model](https://github.com/csdms/bmi-example-python/blob/master/heat/heat.py) and its [BMI](https://github.com/csdms/bmi-example-python/blob/master/heat/bmi_heat.py) on GitHub.

Start by importing `os`, `numpy` and the `Heat` BMI:

In [1]:
import os
import numpy as np

from heat import BmiHeat

Create an instance of the model's BMI.

In [2]:
x = BmiHeat()

What's the name of this model?

In [3]:
print(x.get_component_name())

The 2D Heat Equation


Start the `Heat` model through its BMI using a configuration file:

In [4]:
cat heat.yaml

# Heat model configuration
shape:
  - 6
  - 8
spacing:
  - 1.0
  - 1.0
origin:
  - 0.0
  - 0.0
alpha: 1.0


In [5]:
x.initialize('heat.yaml')

Check the time information for the model.

In [6]:
print('Start time:', x.get_start_time())
print('End time:', x.get_end_time())
print('Current time:', x.get_current_time())
print('Time step:', x.get_time_step())
print('Time units:', x.get_time_units())

Start time: 0.0
End time: 1.7976931348623157e+308
Current time: 0.0
Time step: 0.25
Time units: s


Show the input and output variables for the component (aside on [Standard Names](https://csdms.colorado.edu/wiki/CSDMS_Standard_Names)):

In [7]:
print(x.get_input_var_names())
print(x.get_output_var_names())

('plate_surface__temperature',)
('plate_surface__temperature',)


Next, get the identifier for the grid on which the temperature variable is defined:

In [8]:
grid_id = x.get_var_grid('plate_surface__temperature')
print('Grid id:', grid_id)

Grid id: 0


Then get the grid attributes:

In [9]:
print('Grid type:', x.get_grid_type(grid_id))

rank = x.get_grid_rank(grid_id)
print('Grid rank:', rank)

shape = np.ndarray(rank, dtype=int)
x.get_grid_shape(grid_id, shape)
print('Grid shape:', shape)

spacing = np.ndarray(rank, dtype=float)
x.get_grid_spacing(grid_id, spacing)
print('Grid spacing:', spacing)

Grid type: uniform_rectilinear
Grid rank: 2
Grid shape: [6 8]
Grid spacing: [1. 1.]


These commands are made somewhat un-Pythonic by the generic design of the BMI.

Through the model's BMI, zero out the initial temperature field, except for an impulse near the middle.
Note that *set_value* expects a one-dimensional array for input.

In [10]:
temperature = np.zeros(shape)
temperature[3, 4] = 100.0
x.set_value('plate_surface__temperature', temperature)

Check that the temperature field has been updated. Note that *get_value* expects a one-dimensional array to receive output.

In [11]:
temperature_flat = np.empty_like(temperature).flatten()
x.get_value('plate_surface__temperature', temperature_flat)
print(temperature_flat.reshape(shape))

[[  0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0. 100.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.]]


Now advance the model by a single time step:

In [12]:
x.update()

View the new state of the temperature field:

In [13]:
x.get_value('plate_surface__temperature', temperature_flat)
print(temperature_flat.reshape(shape))

[[ 0.   0.   0.   0.   0.   0.   0.   0. ]
 [ 0.   0.   0.   0.   0.   0.   0.   0. ]
 [ 0.   0.   0.   0.  12.5  0.   0.   0. ]
 [ 0.   0.   0.  12.5 50.  12.5  0.   0. ]
 [ 0.   0.   0.   0.  12.5  0.   0.   0. ]
 [ 0.   0.   0.   0.   0.   0.   0.   0. ]]


There's diffusion!

Advance the model to some distant time:

In [14]:
distant_time = 2.0
while x.get_current_time() < distant_time:
    x.update()

View the final state of the temperature field:

In [15]:
np.set_printoptions(formatter={'float': '{: 5.1f}'.format})
x.get_value('plate_surface__temperature', temperature_flat)
print(temperature_flat.reshape(shape))

[[  0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0]
 [  0.0   0.2   0.9   2.1   2.8   2.1   0.9   0.0]
 [  0.0   0.7   2.2   4.7   6.2   4.7   2.1   0.0]
 [  0.0   0.9   3.0   6.1   7.9   6.1   2.8   0.0]
 [  0.0   0.6   2.0   4.1   5.3   4.1   1.8   0.0]
 [  0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0]]


Note that temperature isn't conserved on the plate:

In [16]:
print(temperature_flat.sum())

74.10263419151306


End the model:

In [17]:
x.finalize()